# Submit an experiment config to a queue to be run

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.text import *
import pdb

In [3]:
import ulmfit_experiments.experiments as experiments
import ulmfit_experiments.classifiers as classifiers
import ulmfit_experiments.sequence_aggregations as sequence_aggregations
from ulmfit_experiments.experiments import * 
from ulmfit_experiments.runner import MongoExperimentRunner

In [4]:
runner = MongoExperimentRunner(ExperimentCls.factory, '/home/tomasz/nlp/trained_models/')

In [50]:
phases = [{'freeze_to': -1, 'cyc_len': 1, 'lr_max_last': 1e-2},
    {'freeze_to': -2, 'cyc_len': 1, 'lr_max_last': 1e-2},
    {'freeze_to': -3, 'cyc_len': 1, 'lr_max_last': 5e-3},
    {'freeze_to': -6, 'cyc_len': 10, 'lr_max_last': 1e-3}]
config = {
    'type': 'Poleval1',
    'params': {
        'dataset_path': '/home/tomasz/nlp/poleval/csv/',
        'encoder_subdir': 'encoder_tweets_spm_basic',
        'training_phases': phases,
        'aggregation_class': 'BranchingAttentionAggregation',
        'aggregation_params': {'att_hid_layers': [50], 'att_dropouts': 0.2, 'agg_dim': 50},
        'cv_fold_num': 0,
        'drop_mult': .5,
#         'train_set_fraction': 0.01,
        'rnn_output_layers': [-1],
        'bidir': False,
        'max_len': 70*20,
        'bs': 80,
        'metrics': ['accuracy', 'f1'],
        'callbacks': ['average_precision_score'],
        'calc_test_score': False
    }
}

runner.submit_job(config)

In [9]:
ex = experiments.ExperimentCls.factory('Poleval1', config['params'])

In [11]:
# results, learn = ex.run()

# Re-running experiments with modified params

In [8]:
db = runner._get_db()

In [30]:
source_id = 83

In [31]:
source_config = db[runner.completed_collection].find_one(source_id)

In [32]:
c = {}
c['type'] = source_config['type']
c['params'] = source_config['params']

In [33]:
for f in range(1, 5):
    c['params']['cv_fold_num'] = f
    runner.submit_job(c)

## From failed

In [35]:
# for source_id in range(7, 12):
#     source_config = db[runner.failed_collection].find_one(source_id)
#     c = {}
#     c['type'] = source_config['type']
#     c['params'] = source_config['params']
#     c['params']['max_len'] = 700
#     runner.submit_job(c)